# Pipeline for Recovering Neural Activity from Calcium Imaging Data

### Relevant Neuroscience Citations / Dependencies
**Motion Correction ->** Suite2P Package from Stringer Lab <br>
**Denoising ->** DeepCAD Package from Dai Lab <br>

**ROI Identification ->** Cellpose Package from Stringer Lab <br>
**Signal Extraction ->** FISSA Package from Rochefort Lab <br>
**Source-Separation ->** FISSA Package from Rochefort Lab <br>
**Spike Inference ->** Cascade Package from Helmchen Lab <br>

## Imports

In [1]:
from ExperimentManagement.ExperimentHierarchy import ExperimentData
import numpy as np

## Generation of Hierarchy

In [2]:
# EH = ExperimentData(Mouse="EM0137", Directory="D:\\EM0137")
# EH.create()

## Load Hierarchy

In [3]:
EH = ExperimentData.loadHierarchy("D:\\EM0137")

Loading Experimental Hierarchy...
Finished.
Activating auto-logging. Current session state plus future input saved.
Filename       : D:\EM0137\log_file.log
Mode           : append
Output logging : True
Raw input log  : True
Timestamping   : True
State          : active
Logging Initiated


Imports

In [4]:
from ImagingAnalysis.PreprocessingImages import PreProcessing

Construct Folder for Data

In [6]:
try:
    EH.Encoding.addImageSamplingFolder(10)
except FileExistsError:
    print("Already created folders")

The sampling folder already exists. Adding to folder dictionary
Already created folders


Define Directories

In [8]:
from ExperimentManagement.ExperimentHierarchy import CollectedDataFolder
EH.Encoding.folder_dictionary["raw_imaging_data"] = CollectedDataFolder(EH.Encoding.folder_dictionary.get("raw_imaging_data"))
RawVideoDirectory = EH.Encoding.folder_dictionary.get("raw_imaging_data").path
OutputDirectory = EH.Encoding.folder_dictionary.get("imaging_10Hz").folders.get("compiled")

Repackage bruker files into a single tiff

In [9]:
PreProcessing.repackage_bruker_tiffs(RawVideoDirectory, OutputDirectory)
EH.Encoding.update_folder_dictionary()

Repackaging...


Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

...

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/5597 [00:00<?, ?it/s]

Finished Repackaging Bruker Tiffs


Load Images

In [10]:
images = PreProcessing.load_all_tiffs(OutputDirectory)

Loading Images...:   0%|          | 0/16 [00:00<?, ?it/s]

Filter Images

In [11]:
images = PreProcessing.blockwise_fast_filter_tiff(images, Footprint=np.ones((7, 3, 3)))

Filtering Images...:   0%|          | 0/6 [00:00<?, ?it/s]

Grouped Z-Project

In [12]:
# for spatial complexity purposes I saved to binary and worked on the memory mapped file, ignore this cell
images = PreProcessing.grouped_z_project(images, (3, 1, 1), np.mean)

Downsampling data...


MemoryError: Unable to allocate 72.0 GiB for an array with shape (36866, 512, 512) and data type float64

Remove Shutter Artifact

In [ ]:
images = PreProcessing.remove_shuttle_artifact(images, chunk_size=7000, artifact_length=1000)

Store Parameters

In [ ]:
Parameters = {
    "grouped-z project bin size": 3,
    "median filter tensor size": (7, 3, 3),
    "shuttle artifact length": 1000,
    "chunk size": 7000,
}
EH.Encoding.folder_dictionary.get("imaging_10Hz").add_notes("preprocessing", Parameters)

Save Images as Binary

In [ ]:
PreProcessing.save_raw_binary(images, OutputDirectory)
EH.Encoding.update_folder_dictionary()
EH.recordMod("Repackaged, filtered, and exported images as raw binary. Made video even length this time")

Cleanup

In [ ]:
EH.Encoding.folder_dictionary.get("imaging_10Hz").clean_up_compilation()
del images

## Image Processing

Motion Correction

In [ ]:
from ImagingAnalysis.Suite2PAnalysis import Suite2PModule

In [ ]:
S2P = Suite2PModule(EH.Encoding.folder_dictionary.get("imaging_10Hz").folders.get("compiled"), EH.Encoding.folder_dictionary.get("imaging_10Hz").path, file_type="binary")
S2P.motionCorrect()

ROI Detection & Classification

In [1]:
S2P.ops["meanImg_chan2"] = np.array([0])
S2P.ops.pop("meanImg_chan2")
S2P.db  = S2P.ops
S2P.roiDetection()
S2P.extractTraces()
S2P.classifyROIs()
S2P.spikeExtraction() # Finalize (Required spks.npy to use GUI)

NameError: name 'np' is not defined

In [ ]:
S2P.iscell, S2P.stat = S2P.remove_small_neurons(S2P.iscell, S2P.stat)
S2P.save_files()
EH.recordStageMod("Encoding", "S2P Encoding")
EH.Encoding.update_folder_dictionary()
EH.saveHierarchy()
del S2P

Fissa: Signal Extraction

In [ ]:
from ImagingAnalysis.FissaAnalysis import FissaAnalysis

In [ ]:
Image = np.reshape(np.fromfile(EH.Encoding.folder_dictionary.get("imaging_10Hz").find_matching_files("registered_data.bin", "plane0")[0], dtype=np.int16), (-1, 512, 512))
PreProcessing.save_raw_binary(Image, EH.Encoding.folder_dictionary.get("imaging_10Hz").folders.get("denoised"))
# saved reg to denoised

In [ ]:
Fissa = FissaAnalysis(data_folder=EH.Encoding.folder_dictionary.get("imaging_10Hz").path, video_folder=EH.Encoding.folder_dictionary.get("imaging_10Hz").folders.get("denoised"), output_folder=EH.Encoding.folder_dictionary.get("imaging_10Hz").folders.get("fissa"))
Fissa.initializeFissa()

In [ ]:
Fissa.extractTraces() # simple, call to extract raw traces from videos
Fissa.saveFissaPrep()

Trace Processing

In [ ]:
from ImagingAnalysis.DataProcessing import Processing

In [ ]:
# let's smooth the data with edge-preserving to make it nicer
#Fissa.ProcessedTraces.smoothed_raw = Processing.smoothTraces_TiffOrg(Fissa.preparation.raw, niter=5, kappa=0.15, gamma=0.15)[0]
#Fissa.preparation.raw = Fissa.ProcessedTraces.smoothed_raw.copy()
#Let's use for separation, so replace the raws with smooths
Fissa.passPrepToFissa()

Fissa: Source-Separation

In [ ]:
Fissa.separateTraces() # simple, call to separate the traces
Fissa.saveFissaSep()

Trace Post-Processing

In [ ]:
# Smooth
Fissa.ProcessedTraces.smoothed_result = Processing.smoothTraces_TiffOrg(Fissa.experiment.result, niter=25, kappa=0.15, gamma=0.25)[0]

# Calculate Fo/F
Fissa.ProcessedTraces.dFoF_result = Processing.calculate_dFoF(Fissa.ProcessedTraces.smoothed_result, Fissa.frame_rate, raw=Fissa.preparation.raw, merge_after=False)

# Condense the ROI Traces for each Trial into a Single Matrix
Fissa.ProcessedTraces.merged_dFoF_result = Processing.mergeTraces(Fissa.ProcessedTraces.dFoF_result)

for _trace in range(Fissa.ProcessedTraces.merged_dFoF_result.shape[0]):
    Fissa.ProcessedTraces.merged_dFoF_result[_trace, :] = Fissa.ProcessedTraces.merged_dFoF_result[_trace, :] - np.mean(Fissa.ProcessedTraces.merged_dFoF_result[_trace, :])

# Detrend the Traces by fitting a 4th-order polynomial and subsequently subtracting
Fissa.ProcessedTraces.detrended_merged_dFoF_result = Processing.detrendTraces(Fissa.ProcessedTraces.merged_dFoF_result, order=4, plot=False)

# Save
Fissa.saveProcessedTraces()
EH.recordMod("Encoding", "Encoding Source-Separation")
EH.saveHierarchy()

Cascade: Event/Spike/Firing Rate Inference

In [ ]:
from ImagingAnalysis.CascadeAnalysis import CascadeModule

In [ ]:
Cascade = CascadeModule(Fissa.ProcessedTraces.detrended_merged_dFoF_result, 10, model_folder="C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\Pretrained_models", SavePath=EH.Encoding.folder_dictionary.get("imaging_10Hz").folders.get("cascade"))

In [ ]:
# Pull Available Models
list_of_models = Cascade.pullModels(Cascade.model_folder)
# Select Model: If you know what model you want, you should use the string instead.
# This model is Global_EXC_10Hz_smoothing_100ms
# Cascade.model_name = list_of_models[21]
Cascade.model_name = "Global_EXC_10Hz_smoothing100ms"
Cascade.downloadModel(Cascade.model_name, "C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\Pretrained_models")

In [ ]:
# Infer Spike Probability
Cascade.predictSpikeProb() # Simple, call to infer spike probability for each frame
# Calculate Firing Rates # Simple, firing rate = spike probability * imaging frequency
Cascade.ProcessedInferences.firing_rates = Processing.calculateFiringRate(Cascade.spike_prob, Cascade.frame_rate)

In [ ]:
Cascade.saveSpikeProb()
Cascade.saveProcessedInferences()

In [ ]:
Cascade.inferDiscreteSpikes()

In [ ]:
Cascade.saveSpikeInference()

In [ ]:
EH.recordStageMod("Encoding", "Encoding Cascade")
EH.saveHierarchy()

In [ ]:
EH.Encoding.update_folder_dictionary()
EH.recordStageMod("Encoding", "Ready")
EH.saveHierarchy()
del Cascade

## Import Behavioral Data

In [ ]:
EH.Encoding.__load_base_behavior()
EH.Encoding.__load_dlc_data(5, 799)
EH.Encoding.load_bruker_data(EH.Encoding.folder_dictionary.get("imaging_10Hz").parameters)
EH.Encoding.data = EH.Encoding.__sync_grouped_z_projected_images(EH.Encoding.data, EH.Encoding.meta,
                                             EH.Encoding.folder_dictionary.get("imaging_10Hz").parameters)
EH.recordStageMod("Encoding", "Added Down Sample Sync Encoding")
EH.saveHierarchy()

## Decoding

In [ ]:
firing_rates = EH.Encoding.folder_dictionary.get("imaging_10Hz").import_proc_inferences().firing_rates

In [ ]:
def trial_matrix_org(DataFrame_, NeuralData):
    _trial_index = np.where(DataFrame_[" TrialIndicator"] >= 3.3)[0]
    _trial_frame = DataFrame_.iloc[_trial_index]
    _cut_to_images = _trial_frame[~_trial_frame["Downsampled Imaging Frame"].isnull()]
    _selected_frames = np.unique(_cut_to_images["Downsampled Imaging Frame"].values)
    _cs_index = _cut_to_images["CS"].values
    OrgNeuralData = NeuralData[:, _selected_frames.astype(int)]
    # noinspection PyShadowingNames

    Features = np.full((2, _cs_index.shape[0]), 0, dtype=np.float64)
    Features[0, np.where(_cs_index == 0)[0]] = 1
    Features[1, np.where(_cs_index == 1)[0]] = 1

    return OrgNeuralData, Features

In [ ]:
DataFrame = EH.Encoding.data.copy(deep=True)
cs_ids = EH.Encoding.trial_parameters.get("stimulusTypes")

In [ ]:
from ImagingAnalysis.DataProcessing import Processing

In [ ]:
NFR = Processing.normalizeSmoothFiringRates(firing_rates, 5)

In [ ]:
NeuralData_Matrix, FeatureData_Matrix = trial_matrix_org(DataFrame, NFR)

In [ ]:
from ComputationalAnalysis.SupportVectorMachine import SVM

In [ ]:
SVR = SVM(NeuralData=NeuralData_Matrix, FeatureData=FeatureData_Matrix)


In [ ]:
SVR.neural_data, SVR.feature_data = SVR.shuffleFrames(SVR.neural_data, FeatureData=SVR.feature_data)
SVR.feature_data = SVR.feature_data[0, :]

In [ ]:
SVR.splitData()

In [ ]:
SVR.fitModel()
SVR.assessFit()
SVR.makeAllPredictions()
SVR.commonAssessment()